# RGB a YCbCr

In [ ]:
import numpy as np

def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

def ycbcr2rgb(im):
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(np.float)
    rgb[:,:,[1,2]] -= 128
    rgb = rgb.dot(xform.T)
    np.putmask(rgb, rgb > 255, 255)
    np.putmask(rgb, rgb < 0, 0)
    return np.uint8(rgb)

In [ ]:
!wget https://cdn.foodandwineespanol.com/2021/01/manzanas-licor-de-manzana.jpg -O image.jpg

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imshow
img = imread('image.jpg')
imshow(img)

In [ ]:
img = rgb2ycbcr(img)
imshow(img)

# DTC

In [ ]:
import numpy as np
from scipy.fftpack import dct

def dct2D(x):
	tmp = dct(x, type=2 ,norm='ortho').transpose()
	return dct(tmp, type=2 ,norm='ortho').transpose()

print(dct2D([
	     [-64., -68., -71., -72., -80., -81., -81., -85.],
	     [-67., -70., -75., -76., -80., -79., -76., -75.],
	     [-61., -68., -75., -75., -79., -81., -80., -74.],
	     [-60., -67., -65., -65., -66., -63., -63., -64.],
	     [-57., -67., -58., -65., -59., -54., -40., -40.],
	     [-45., -36., -26., -23., -21., -17., -18., -13.],
	     [-33., -20., -20., -4., -6., 2., 0., 0.],
	     [-21., -10., -3., 6., 9., 14., 13., 9.]
	    ]))

# Ejemplo:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#This line is required to display visualizations in the browser
%matplotlib inline

In [ ]:
def mc_pi(ntrials):
    """
    calculate the value of pi using montecarlo method and visualize the process

    """
    x = np.random.random(ntrials)
    y = np.random.random(ntrials)
    #masking
    inside_circle = x**2+y**2 < 1
    unit_circle_x = np.linspace(0,1,100)
    unit_circle = [unit_circle_x,np.sqrt(1.0-unit_circle_x**2)]
    plt.plot(*unit_circle,color='black')
    plt.scatter(x[inside_circle],y[inside_circle],marker='.',color='blue',s=1)
    plt.scatter(x[~inside_circle],y[~inside_circle],marker='.',color='red',s=1)
    plt.title("value of $\pi$="+str( 4.0*np.sum(inside_circle)/float(ntrials)))

In [ ]:
from ipywidgets import interact,interactive,interact_manual
mc_widget=interactive(mc_pi,ntrials=(1,100000,10));
mc_widget

# skimage image processing library

In [ ]:
from skimage import data
from skimage.color import rgb2gray

In [ ]:
from skimage import img_as_ubyte,img_as_float
gray_images = {
        "cat":rgb2gray(img_as_float(data.chelsea())),
        "astro":rgb2gray(img_as_float(data.astronaut())),
        "camera":data.camera(),
        "coin": data.coins(),
        "clock":data.clock(),
        "blobs":data.binary_blobs(),
        "coffee":rgb2gray(img_as_float(data.coffee()))
}

# svd in python

Resultados de traducción
Usaremos la función svd de la biblioteca numpy.linalg para calcular svd de una matriz en python. La función svd devuelve U,s,V .


* U ha dejado vectores singulares en las columnas

* s es una matriz numpy de rango 1 con valores singulares

* V tiene vectores rectos singulares en las filas, equivalentes al álgebra lineal tradicional

In [ ]:
from numpy.linalg import svd

In [ ]:
def compress_svd(image,k):
    """
    Perform svd decomposition and truncated (using k singular values/vectors) reconstruction
    returns
    --------
      reconstructed matrix reconst_matrix, array of singular values s
    """
    U,s,V = svd(image,full_matrices=False)
    reconst_matrix = np.dot(U[:,:k],np.dot(np.diag(s[:k]),V[:k,:]))

    return reconst_matrix,s

# Compress gray scale images

In [ ]:
def compress_show_gray_images(img_name,k):
    """
     compresses gray scale images and display the reconstructed image.
     Also displays a plot of singular values
    """
    image=gray_images[img_name]
    original_shape = image.shape
    reconst_img,s = compress_svd(image,k)
    fig,axes = plt.subplots(1,2,figsize=(8,5))
    axes[0].plot(s)
    compression_ratio =100.0* (k*(original_shape[0] + original_shape[1])+k)/(original_shape[0]*original_shape[1])
    axes[1].set_title("compression ratio={:.2f}".format(compression_ratio)+"%")
    axes[1].imshow(reconst_img,cmap='gray')
    axes[1].axis('off')
    fig.tight_layout()

In [ ]:
def compute_k_max(img_name):
  """
    utility function for calculating max value of the slider range
  """
  img = gray_images[img_name]
  m,n = img.shape
  return m*n/(m+n+1)

#set up the widgets
import ipywidgets as widgets

list_widget = widgets.Dropdown(options=list(gray_images.keys()))
int_slider_widget = widgets.IntSlider(min=1,max=compute_k_max('cat'))
def update_k_max(*args):
  img_name=list_widget.value
  int_slider_widget.max = compute_k_max(img_name)
list_widget.observe(update_k_max,'value')


In [ ]:
interact(compress_show_gray_images,img_name=list_widget,k=int_slider_widget);

In [ ]:
color_images = {
    "cat":img_as_float(data.chelsea()),
    "astro":img_as_float(data.astronaut()),
    "coffee":img_as_float(data.coffee())

}

# Compress color images

In [ ]:
def compress_show_color_images_reshape(img_name,k):
    """
     compress and display the reconstructed color image using the reshape method
    """
    image = color_images[img_name]
    original_shape = image.shape
    image_reshaped = image.reshape((original_shape[0],original_shape[1]*3))
    image_reconst,_ = compress_svd(image_reshaped,k)
    image_reconst = image_reconst.reshape(original_shape)
    compression_ratio =100.0* (k*(original_shape[0] + 3*original_shape[1])+k)/(original_shape[0]*original_shape[1]*original_shape[2])
    plt.title("compression ratio={:.2f}".format(compression_ratio)+"%")
    plt.imshow(image_reconst)

In [ ]:
def compute_k_max_color_images(img_name):
  image = color_images[img_name]
  original_shape = image.shape
  return (original_shape[0]*original_shape[1]*original_shape[2])//(original_shape[0] + 3*original_shape[1] + 1)


list_widget = widgets.Dropdown(options=list(color_images.keys()))
int_slider_widget = widgets.IntSlider(min=1,max=compute_k_max_color_images('cat'))
def update_k_max_color(*args):
  img_name=list_widget.value
  int_slider_widget.max = compute_k_max_color_images(img_name)
list_widget.observe(update_k_max_color,'value')

In [ ]:
interact(compress_show_color_images_reshape,img_name=list_widget,k=int_slider_widget);


# Layers method to compress color images

In [ ]:
def compress_show_color_images_layer(img_name,k):
    """
     compress and display the reconstructed color image using the layer method
    """
    image = color_images[img_name]
    original_shape = image.shape
    image_reconst_layers = [compress_svd(image[:,:,i],k)[0] for i in range(3)]
    image_reconst = np.zeros(image.shape)
    for i in range(3):
        image_reconst[:,:,i] = image_reconst_layers[i]

    compression_ratio =100.0*3* (k*(original_shape[0] + original_shape[1])+k)/(original_shape[0]*original_shape[1]*original_shape[2])
    plt.title("compression ratio={:.2f}".format(compression_ratio)+"%")

    plt.imshow(image_reconst)

In [ ]:
def compute_k_max_color_images_layers(img_name):
  image = color_images[img_name]
  original_shape = image.shape
  return (original_shape[0]*original_shape[1]*original_shape[2])// (3*(original_shape[0] + original_shape[1] + 1))


list_widget = widgets.Dropdown(options=list(color_images.keys()))
int_slider_widget = widgets.IntSlider(min=1,max=compute_k_max_color_images_layers('cat'))
def update_k_max_color_layers(*args):
  img_name=list_widget.value
  int_slider_widget.max = compute_k_max_color_images_layers(img_name)
list_widget.observe(update_k_max_color_layers,'value')

In [ ]:
interact(compress_show_color_images_layer,img_name=list_widget,k=int_slider_widget);
